In [ ]:
%load_ext google.colab.data_table

In [39]:
# Question perso :

# Comment les conditions météorologiques prévues à Paris en 2024 peuvent-elles influencer les performances dans certaines disciplines ?
# Quels sont les progrès ou régressions des performances des athlètes clés dans les compétitions récentes (championnats mondiaux, compétitions nationales) ?
# Quels athlètes ont été récemment blessés et comment cela pourrait-il affecter leur performance ?
# Y a-t-il de nouveaux athlètes prometteurs qui pourraient surprendre et surpasser les favoris ?
# Les athlètes français bénéficieront-ils d’un avantage significatif en jouant à domicile ?
# Comment l’expérience passée des athlètes aux Jeux Olympiques influence-t-elle leurs performances futures ?
# Ukraine-Russie. La situation géopolitique peut elle affecter la préparation des athlètes, leur moral, et même leur participation ?
# Le sentiment et l’état émotionnel des athlètes peuvent influencer leurs performances ?
# Y a-t-il une corrélation entre le nombre de médailles remportées et des facteurs économiques tel que le montant du Produit Intérieur Brut (PIB) ?

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=f4e3b6098b14ffd9060343b4b5678effcd98ef595ab84e4d44e76b8ab7766af3
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [43]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import udf
from pyspark.sql.window import Window
from pyspark.conf import SparkConf
import pandas as pd
import plotly.express as px
from pyspark.sql.functions import col, count, when

In [5]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
spark

In [16]:
excel_file_path = "olympic_medals.xlsx"
df = pd.read_excel(excel_file_path)
csv_file_path = "./yourfile.csv"
df.to_csv(csv_file_path, index=False)

In [41]:
df_medals = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("./olympic_medals.csv")
df_medals.show()

+----------+----------------+------------+--------------------+------------+----------+----------------+-----------------+--------------------+--------------------+--------------------+------------+---------------------+
|Unnamed: 0|discipline_title|   slug_game|         event_title|event_gender|medal_type|participant_type|participant_title|         athlete_url|   athlete_full_name|        country_name|country_code|country_3_letter_code|
+----------+----------------+------------+--------------------+------------+----------+----------------+-----------------+--------------------+--------------------+--------------------+------------+---------------------+
|         0|         Curling|beijing-2022|       Mixed Doubles|       Mixed|      GOLD|        GameTeam|            Italy|https://olympics....|Stefania CONSTANTINI|               Italy|          IT|                  ITA|
|         1|         Curling|beijing-2022|       Mixed Doubles|       Mixed|      GOLD|        GameTeam|            

In [24]:
excel_file_path = "olympic_athletes.json"
df = pd.read_json(excel_file_path)
csv_file_path = "./olympic_athletes.csv"
df.to_csv(csv_file_path, index=False)

In [42]:
df_athletes = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("./olympic_athletes.csv")
df_athletes.show()

+--------------------+--------------------+--------------------+----------------+------------------+--------------+----+
|         athlete_url|   athlete_full_name|games_participations|      first_game|athlete_year_birth|athlete_medals| bio|
+--------------------+--------------------+--------------------+----------------+------------------+--------------+----+
|https://olympics....|Cooper WOODS-TOPA...|                   1|    Beijing 2022|            2000.0|          NULL|NULL|
|https://olympics....|      Felix ELOFSSON|                   2|PyeongChang 2018|            1995.0|          NULL|NULL|
|https://olympics....|       Dylan WALCZYK|                   1|    Beijing 2022|            1993.0|          NULL|NULL|
|https://olympics....|       Olli PENTTALA|                   1|    Beijing 2022|            1995.0|          NULL|NULL|
|https://olympics....|    Dmitriy REIKHERD|                   1|    Beijing 2022|            1989.0|          NULL|NULL|
|https://olympics....|         M

In [27]:
excel_file_path = "olympic_results.html"
df = pd.read_html(excel_file_path)
csv_file_path = "./olympic_results.csv"
df.to_csv(csv_file_path, index=False)

AttributeError: 'list' object has no attribute 'to_csv'

In [28]:
html_file_path = "olympic_results.html"
dfs = pd.read_html(html_file_path)
print(f"Nombre de tables trouvées : {len(dfs)}")
df = dfs[0]
csv_file_path = "./olympic_results.csv"
df.to_csv(csv_file_path, index=False)

Nombre de tables trouvées : 1
Fichier CSV sauvegardé à l'emplacement : ./olympic_results.csv


In [40]:
df_results = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("./olympic_results.csv")
df_results.show()

+----------+----------------+-------------+------------+----------------+----------+--------------------+----------+-------------+--------------------+------------+---------------------+-----------+-----------------+----------+----------+
|Unnamed: 0|discipline_title|  event_title|   slug_game|participant_type|medal_type|            athletes|rank_equal|rank_position|        country_name|country_code|country_3_letter_code|athlete_url|athlete_full_name|value_unit|value_type|
+----------+----------------+-------------+------------+----------------+----------+--------------------+----------+-------------+--------------------+------------+---------------------+-----------+-----------------+----------+----------+
|         0|         Curling|Mixed Doubles|beijing-2022|        GameTeam|      GOLD|[('Stefania CONST...|     False|            1|               Italy|          IT|                  ITA|       NULL|             NULL|      NULL|      NULL|
|         1|         Curling|Mixed Doubles|b

In [30]:
excel_file_path = "olympic_hosts.xml"
df = pd.read_xml(excel_file_path)
csv_file_path = "./olympic_hosts.csv"
df.to_csv(csv_file_path, index=False)

In [34]:
df_data_olympic_hosts = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("./olympic_hosts.csv")
df_data.show()

+-----+-------------------+-------------------+-------------------+------------------+-------------------+-----------+---------+
|index|          game_slug|      game_end_date|    game_start_date|     game_location|          game_name|game_season|game_year|
+-----+-------------------+-------------------+-------------------+------------------+-------------------+-----------+---------+
|    0|       beijing-2022|2022-02-20 12:00:00|2022-02-04 15:00:00|             China|       Beijing 2022|     Winter|     2022|
|    1|         tokyo-2020|2021-08-08 14:00:00|2021-07-23 11:00:00|             Japan|         Tokyo 2020|     Summer|     2020|
|    2|   pyeongchang-2018|2018-02-25 08:00:00|2018-02-08 23:00:00| Republic of Korea|   PyeongChang 2018|     Winter|     2018|
|    3|           rio-2016|2016-08-21 21:00:00|2016-08-05 12:00:00|            Brazil|           Rio 2016|     Summer|     2016|
|    4|         sochi-2014|2014-02-23 16:00:00|2014-02-07 04:00:00|Russian Federation|         So

In [35]:
# 1. La France a organisé 6 JO : 3 d’hiver et 3 d’été (en comptant celui de 2024) ?

In [36]:
df_france = df_data_olympic_hosts.filter(df_data_olympic_hosts.game_location == "France")
df_france.show()
df_france_pd = df_france.toPandas()
counts_by_season = df_france_pd['game_season'].value_counts().reset_index()
counts_by_season.columns = ['Season', 'Count']

fig = px.bar(counts_by_season, x='Season', y='Count', title='Nombre de Jeux Olympiques organisés en France par saison')
fig.show()

+-----+----------------+-------------------+-------------------+-------------+----------------+-----------+---------+
|index|       game_slug|      game_end_date|    game_start_date|game_location|       game_name|game_season|game_year|
+-----+----------------+-------------------+-------------------+-------------+----------------+-----------+---------+
|   16|albertville-1992|1992-02-23 19:00:00|1992-02-08 07:00:00|       France|Albertville 1992|     Winter|     1992|
|   28|   grenoble-1968|1968-02-18 19:00:00|1968-02-06 07:00:00|       France|   Grenoble 1968|     Winter|     1968|
|   45|      paris-1924|1924-07-27 19:00:00|1924-05-04 07:00:00|       France|      Paris 1924|     Summer|     1924|
|   46|   chamonix-1924|1924-02-05 20:00:00|1924-01-25 08:00:00|       France|   Chamonix 1924|     Winter|     1924|
|   51|      paris-1900|1900-10-28 19:50:39|1900-05-14 08:50:39|       France|      Paris 1900|     Summer|     1900|
+-----+----------------+-------------------+------------

In [ ]:
# 2. La France est le 2è pays qui a organisé le plus de JO après les USA (8 JO) ?

In [37]:
df_host_counts = df_data_olympic_hosts.groupBy("game_location").count().orderBy("count", ascending=False)
df_host_counts.show()
df_host_counts_pd = df_host_counts.toPandas()
fig = px.bar(df_host_counts_pd, x='game_location', y='count', title='Nombre de Jeux Olympiques organisés par pays')
fig.show()

+--------------------+-----+
|       game_location|count|
+--------------------+-----+
|       United States|    8|
|              France|    5|
|               Japan|    4|
|       Great Britain|    3|
|               Italy|    3|
|              Canada|    3|
|   Republic of Korea|    2|
|             Germany|    2|
|              Greece|    2|
|               China|    2|
|              Norway|    2|
|         Switzerland|    2|
|             Austria|    2|
|              Sweden|    1|
|             Belgium|    1|
|             Finland|    1|
|               Spain|    1|
|  Russian Federation|    1|
|Federal Republic ...|    1|
|              Mexico|    1|
+--------------------+-----+
only showing top 20 rows



In [ ]:
# 3. Les JO d’hiver sont nés à Chamonix en 1924 ?

In [38]:
df_chamonix_1924 = df_data_olympic_hosts.filter(
    (df_data_olympic_hosts.game_location == "France") &
    (df_data_olympic_hosts.game_name == "Chamonix 1924") &
    (df_data_olympic_hosts.game_season == "Winter")
)

df_chamonix_1924.show()

+-----+-------------+-------------------+-------------------+-------------+-------------+-----------+---------+
|index|    game_slug|      game_end_date|    game_start_date|game_location|    game_name|game_season|game_year|
+-----+-------------+-------------------+-------------------+-------------+-------------+-----------+---------+
|   46|chamonix-1924|1924-02-05 20:00:00|1924-01-25 08:00:00|       France|Chamonix 1924|     Winter|     1924|
+-----+-------------+-------------------+-------------------+-------------+-------------+-----------+---------+



In [ ]:
# 4. JO de Paris, en 1900 : les femmes peuvent participer aux JO ?


In [ ]:
# 5. Seuls 4 athlètes ont remporté des médailles à la fois aux JO d’hiver et d’été. Une seule d’entre eux, Christa Ludinger-Rothenburger, a remporté des médailles au cours de la même année ?


In [ ]:
# 6. De 1924 à 1992, les JO d’hiver et d’été avaient lieu au cours de la même année. Désormais, ils sont organisés selon des cycles distincts avec une alternance de 2 ans ?


In [ ]:
# 7. Tarzan lui-même a participé aux JO. En effet, Johnny Weissmuller, ancien athlète devenu acteur et ayant interprété 12 films de Tarzan, a remporté 5 médailles d’or en natation dans les années 1920 ?


In [ ]:
# 8. Les JO de Londres de 2012 ont été les 1ers JO durant lesquels tous les pays participants ont envoyé des athlètes de sexe féminin ?


In [ ]:
# 9. Les sports suivants ne font (malheureusement) plus partie des J.O : la natation synchronisée en solo, le tir à la corde, la corde à grimper, la montgolfière, le duel au pistolet, le vélo tandem, la course d’obstacles à la nage et le plongeon à distance. Par chance, le tir au pigeon n’a été mis en place qu’une seule fois pendant les Jeux Olympiques de Paris de 1900 ?


In [ ]:
# 10. Les Jeux de 2016, à Rio, marqueront la 1è manifestation des JO en Amérique du Sud ?


In [ ]:
# 11. Pendant les 17 jours des JO d’été de 2016, 10 500 athlètes, originaires de 205 pays, représenteront 42 sports différents et participeront à 306 épreuves, à Rio ?


In [ ]:
# 1. Combien de médailles la France a remporté : en tout, en Or, en argent et en Bronze (depuis le début des JO) ?


In [47]:
df_france_medals = df_results.filter((col("country_3_letter_code") == "FRA") & (col("medal_type").isNotNull()))
medals_count = df_france_medals.groupBy("medal_type").agg(count("*").alias("Count"))
medals_count.show()
medals_count_pd = medals_count.toPandas()

fig = px.bar(medals_count_pd, x='medal_type', y='Count', title='Nombre de médailles remportées par la France')
fig.show()

+----------+-----+
|medal_type|Count|
+----------+-----+
|    SILVER|  172|
|      GOLD|  154|
|    BRONZE|  217|
+----------+-----+



In [ ]:
# 2. Lors de quelle JO la France a eu le plus (le moins) de succès ?

In [49]:
df_france_medals = df_results.filter((col("country_3_letter_code") == "FRA") & (col("medal_type").isNotNull()))
medals_count_per_edition = df_france_medals.groupBy("slug_game").agg(count("*").alias("Total_Medals"))
medals_count_per_edition.show()
most_successful = medals_count_per_edition.orderBy(col("Total_Medals").desc()).first()
least_successful = medals_count_per_edition.orderBy(col("Total_Medals").asc()).first()

print(f"L'edition avec le plus de succès pour la france : {most_successful['slug_game']} avec {most_successful['Total_Medals']} médailles")
print(f"L'édition avec le moins de succès pour la france : {least_successful['slug_game']} avec {least_successful['Total_Medals']} médailles")

medals_count_per_edition_pd = medals_count_per_edition.toPandas()


fig = px.bar(medals_count_per_edition_pd, x='slug_game', y='Total_Medals', title='Nombre de médailles remportées par la France par édition des JO')
fig.show()

+-------------------+------------+
|          slug_game|Total_Medals|
+-------------------+------------+
|      sarajevo-1984|           3|
|         sochi-2014|          15|
|        nagano-1998|           8|
|       beijing-2022|          14|
|   mexico-city-1968|          15|
|      montreal-1976|           9|
|        munich-1972|          13|
|        athens-2004|          33|
|salt-lake-city-2002|          11|
|        london-2012|          35|
|         turin-2006|           9|
|        moscow-1980|          14|
|     vancouver-2010|          11|
|       beijing-2008|          41|
|  squaw-valley-1960|           3|
|         tokyo-2020|          33|
|     innsbruck-1976|           1|
|   los-angeles-1984|          28|
|         seoul-1988|          16|
|       atlanta-1996|          37|
+-------------------+------------+
only showing top 20 rows

L'edition avec le plus de succès pour la france : rio-2016 avec 42 médailles
L'édition avec le moins de succès pour la france : innsbr

In [ ]:
# 3. Peut-on considérer que la France est la grande spécialiste de certaine(s) discipline(s) sportive(s), comparativement aux autres notions ? Laquelle / Lesquelles ?


In [50]:
df_france_medals = df_results.filter((col("country_3_letter_code") == "FRA") & (col("medal_type").isNotNull()))
df_france_discipline_medals = df_france_medals.groupBy("discipline_title").agg(count("*").alias("Total_Medals_France"))
df_all_discipline_medals = df_results.filter(col("medal_type").isNotNull()).groupBy("discipline_title").agg(count("*").alias("Total_Medals_All"))
df_discipline_comparison = df_france_discipline_medals.join(df_all_discipline_medals, on="discipline_title", how="inner")
df_discipline_comparison = df_discipline_comparison.withColumn("Percentage_France", col("Total_Medals_France") / col("Total_Medals_All") * 100)
df_discipline_comparison.orderBy(col("Percentage_France").desc()).show()
df_discipline_comparison_pd = df_discipline_comparison.toPandas()


fig = px.bar(df_discipline_comparison_pd, x='discipline_title', y='Percentage_France', title='Pourcentage de médailles remportées par la France par discipline')
fig.show()

+--------------------+-------------------+----------------+------------------+
|    discipline_title|Total_Medals_France|Total_Medals_All| Percentage_France|
+--------------------+-------------------+----------------+------------------+
|        Canoe Slalom|                 18|             108|16.666666666666664|
|        Rugby Sevens|                  1|               6|16.666666666666664|
|             Fencing|                 69|             453|  15.2317880794702|
|            Biathlon|                 32|             286|11.188811188811188|
|         Cycling BMX|                  2|              18| 11.11111111111111|
|  Equestrian Jumping|                  9|              87|10.344827586206897|
|Cycling Mountain ...|                  4|              39|10.256410256410255|
|       Alpine Skiing|                 45|             448|10.044642857142858|
|       Cycling Track|                 36|             370|  9.72972972972973|
|                Judo|                 57|          

In [ ]:
# 4. Quelles sont les sports les plus dominants dans les JO au fil des ans ?

In [53]:
df_medals_cleaned = df_results.filter(col("medal_type").isNotNull())
df_medals_cleaned = df_medals_cleaned.withColumn("year", col("slug_game").substr(-4, 4).cast("int"))
df_medals_per_discipline_year = df_medals_cleaned.groupBy("discipline_title", "year").agg(count("*").alias("Total_Medals"))
df_medals_per_discipline_year.show()
df_medals_per_discipline_year_pd = df_medals_per_discipline_year.toPandas()


fig = px.bar(df_medals_per_discipline_year_pd, x='year', y='Total_Medals', color='discipline_title',
             title='Nombre de médailles attribuées par discipline au fil des ans',
             labels={'Total_Medals': 'Nombre de Médailles', 'year': 'Année', 'discipline_title': 'Discipline'},
             barmode='stack')

fig.show()

+--------------------+----+------------+
|    discipline_title|year|Total_Medals|
+--------------------+----+------------+
|              Rowing|2012|          42|
|           Bobsleigh|1998|           7|
|   Modern Pentathlon|1968|           6|
|          Water Polo|2016|           6|
|             Archery|2008|          12|
|          Basketball|1992|           6|
|              Boxing|1992|          48|
|       Speed skating|1980|          28|
|                Luge|1976|           9|
|              Diving|2008|          24|
|        Canoe Sprint|2004|          36|
|                Luge|1972|           9|
|           Badminton|2000|          15|
|        Table Tennis|1992|          16|
|Cross Country Skiing|1992|          30|
|            Handball|1980|           6|
|            Football|1976|           3|
|          Ice Hockey|1956|           3|
|             Curling|2018|           9|
|            Softball|2008|           3|
+--------------------+----+------------+
only showing top